In [1]:
#librerias
library(dplyr)
library(lubridate)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘lubridate’

The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union



# Libreta ejecutada en R

Los datos trabjados so obtenidos de forma libre de la página de la <a href='http://www.aire.cdmx.gob.mx/default.php?opc=%27aKBhnmI=%27&opcion=Zw=='> Ciudad de México</a>, se obtienen diferentes indices y valores desde el año 2000 hasta octubre de 2020.

Se descargaron los datos en archivos CSV, cada archivo representa un año de medición. Para este analisis se descagaron datos meteorológicos para obtener información de la temperatura y datos de contaminantes para obtener valores de **NOx, SO2, CO**

In [67]:
#Datos que pueden ser útiles
estaciones <- read.csv('data/cat_estacion.csv')
#parametros <- read.csv('data/cat_parametros.csv')
#unidades <- read.csv('data/cat_unidades.csv')
lestaciones <- as.character(estaciones$Catalogo.de.estaciones[-1])

## Preproceso de datos
Vamos a obtener de cada uno de las vatiables elegidas (NOx, CO, PM2.5,temperatrua) el promedio mesnual para cada estación considerando lo siguiente:
   * Los datos NA serán descartados y el promedio sera evaluado como
       $$
           \frac{1}{m}\sum^{n}(x_i)
       $$
 Considerando $x_t$ el valor de la variable en el tiempo $t$, $n$ la cantidad de valores registrado en el día (Imaginese como el espacio de parámteros,$T$ ),  $m = n -cantidadDeNA$ la cantidad de valores que se tiene si se desprecia los valores NA

In [136]:
filtroNA <- function(df)
{
    aux <- df['cve_parameter'] == 'TMP'
    df <- df[aux,]
    df[is.na(df)] <- -1
    aux <- df['value'] != -1
    df <- df[aux,]
    df
}
describe <- function(df,lestaciones,year)
{
    df['date'] <- df %>% select(date) %>% mutate(date = dmy_hms(as.character(date)))
    #aux <- df %>% mutate(month = month(date), day = day(date)) %>% group_by(month)
    aux <- df %>% mutate(month = month(date)) %>% group_by(month)
    for(station in lestaciones)   
    {
        print(station)
        print(filter(aux,cve_station == station))
        if(nrow(filter(aux,cve_station == station)) < 1)
        {
            print("pass")
        }
        else
        {
            aux <- filter(aux,cve_station == station) %>% summarise(promedio = mean(value), variance = var(value),min(value),max(value))
            print(aux)
            write.csv(aux, file=paste0("data/transfData/",station,"_",year,"_tmp",".csv"))
        }
    }
}

In [138]:
#Variables meteorológicas, temperatura
#Vamos a leer todos los datos que tenemos
years = 2000:2020
for(year in years)
{
    print(year)
    dm <- read.csv(paste0('data/meteorología_',year,'.csv'),skip = 10, header = T)
    describe(filtroNA(dm),lestaciones,year)
}


[1] 2000
[1] "ACO"
# A tibble: 0 x 6
# Groups:   month [0]
# … with 6 variables: date <dttm>, cve_station <fct>, cve_parameter <fct>,
#   value <dbl>, unit <int>, month <dbl>
[1] "pass"
[1] "AJU"
# A tibble: 0 x 6
# Groups:   month [0]
# … with 6 variables: date <dttm>, cve_station <fct>, cve_parameter <fct>,
#   value <dbl>, unit <int>, month <dbl>
[1] "pass"
[1] "AJM"
# A tibble: 0 x 6
# Groups:   month [0]
# … with 6 variables: date <dttm>, cve_station <fct>, cve_parameter <fct>,
#   value <dbl>, unit <int>, month <dbl>
[1] "pass"
[1] "ARA"
# A tibble: 0 x 6
# Groups:   month [0]
# … with 6 variables: date <dttm>, cve_station <fct>, cve_parameter <fct>,
#   value <dbl>, unit <int>, month <dbl>
[1] "pass"
[1] "ATI"
# A tibble: 0 x 6
# Groups:   month [0]
# … with 6 variables: date <dttm>, cve_station <fct>, cve_parameter <fct>,
#   value <dbl>, unit <int>, month <dbl>
[1] "pass"
[1] "AZC"
# A tibble: 0 x 6
# Groups:   month [0]
# … with 6 variables: date <dttm>, cve_station <fct>, cv

`summarise()` ungrouping output (override with `.groups` argument)


# A tibble: 12 x 5
   month promedio variance `min(value)` `max(value)`
   <dbl>    <dbl>    <dbl>        <dbl>        <dbl>
 1     1     13.9     40.5          1.1         27.8
 2     2     15.5     31.8          3.3         27  
 3     3     17.6     27.6          8           28.1
 4     4     19.1     29.3          8.3         32.5
 5     5     18.7     18.5          8.7         28.7
 6     6     17.5     12.5         11.7         27.9
 7     7     17.9     16.7         11.2         28.1
 8     8     17.0     13.6          9.9         25.6
 9     9     17.5     13.4         10.1         27  
10    10     16.5     16.8          8.5         26.4
11    11     16.6     21.2          8.2         26.8
12    12     13.2     23.5          3.8         24.2
[1] "CFE"


ERROR: Error: Problem with `filter()` input `..1`.
✖ objeto 'cve_station' no encontrado
ℹ Input `..1` is `cve_station == station`.


In [29]:
#dftmp <- dm00 %>% mutate(month = month(date), day = day(date)) %>% group_by(month, day)
dftmp <- dm00[tmp,]
# Creo dos tablas donde almaceno el día y el mes del año (modicar para que quede en una sola colúmna)
# Obtengo una mascara para descartar esos valores para despues obtener los promedios diarios
dftmp[is.na(dftmp)] <- -1
nNA <- dftmp['value'] != -1
dftmp <- dftmp[nNA,]
head(dftmp)

,date,cve_station,cve_parameter,value,unit
122979,2020-01-01 00:01:00,SAG,TMP,10.1,5
122980,2020-01-01 00:01:00,TLA,TMP,9.6,5
122981,2020-01-01 00:01:00,XAL,TMP,10.8,5
122982,2020-01-01 00:01:00,MER,TMP,11.9,5
122983,2020-01-01 00:01:00,PED,TMP,9.8,5
122984,2020-01-01 00:01:00,CES,TMP,9.7,5


In [77]:
#Genero archivos para analizar después
dftmp <- dftmp %>% mutate(month = month(date), day = day(date)) %>% group_by(month)
for(station in lestaciones)   
{
    tmp<-filter(dftmp,cve_station == station) %>% summarise(promedio = mean(value), variance = var(value),min(value),max(value))
    write.csv(tmp, file=paste0(station,"_tmp",".csv"))
}


Warning message in min(value):
“ningún argumento finito para min; retornando Inf”Warning message in max(value):
“ningun argumento finito para max; retornando -Inf”`summarise()` ungrouping output (override with `.groups` argument)
Warning message in min(value):
“ningún argumento finito para min; retornando Inf”Warning message in max(value):
“ningun argumento finito para max; retornando -Inf”`summarise()` ungrouping output (override with `.groups` argument)
Warning message in min(value):
“ningún argumento finito para min; retornando Inf”Warning message in max(value):
“ningun argumento finito para max; retornando -Inf”`summarise()` ungrouping output (override with `.groups` argument)
Warning message in min(value):
“ningún argumento finito para min; retornando Inf”Warning message in max(value):
“ningun argumento finito para max; retornando -Inf”`summarise()` ungrouping output (override with `.groups` argument)
Warning message in min(value):
“ningún argumento finito para min; retornando Inf

In [ ]:
# Mascara de archivos a trabajar
#Contaminantes
dc00 <- read.csv('data/contaminantes_2000.csv',skip = 10, header = T)
nox <- dc00['cve_parameter'] == 'NOX'
co <- dc00['cve_parameter'] == 'CO'
so2 <- dc00['cve_parameter'] == 'SO2'
pm2 <- dc00['cve_parameter'] == 'PM2.5'
dc00['date'] <- dc00 %>% select(date) %>% mutate(date = dmy_hms(as.character(date)))